<a href="https://colab.research.google.com/github/carlaengine/Design-patterns/blob/main/EDAAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Install PyPDF2
!pip install PyPDF2
# Import the display function
from IPython.display import display

# Show the cleaned data


# Import necessary libraries
import pandas as pd
from PyPDF2 import PdfReader
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Function to load PDF data
def load_pdf_data(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        number_of_pages = len(reader.pages)
        pdf_text = ''
        for page in range(number_of_pages):
            pdf_text += reader.pages[page].extract_text()
    return pdf_text

# Function to extract table data from PDF text
def extract_table_data(pdf_text):
    pattern = re.compile(r'(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\w+)')
    matches = pattern.findall(pdf_text)
    return matches

# Load the Excel file
excel_file = '/content/APEX Loan Data (4).xlsx'
df_excel = pd.read_excel(excel_file)

# Load the PDF file
pdf_file = '/content/APEX_Loans_Database_Table (4).pdf'
pdf_text = load_pdf_data(pdf_file)
pdf_data = extract_table_data(pdf_text)

# Create DataFrame from PDF data
columns = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Graduate', 'Self_Employed',
           'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term',
           'Credit_History', 'Property_Area', 'Loan_Status']
df_pdf = pd.DataFrame(pdf_data, columns=columns)

# Convert numeric columns to appropriate types
numeric_columns = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Graduate', 'Self_Employed',
                   'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term',
                   'Credit_History', 'Property_Area']
df_pdf[numeric_columns] = df_pdf[numeric_columns].apply(pd.to_numeric)

# Check for duplicates and remove them
df_pdf = df_pdf.drop_duplicates()
df_excel = df_excel.drop_duplicates()

# Handle missing values only for numeric columns in df_pdf
df_pdf[numeric_columns] = df_pdf[numeric_columns].fillna(df_pdf[numeric_columns].median())

# Handle missing values separately in df_excel
# Numeric columns
numeric_columns_excel = df_excel.select_dtypes(include=['number']).columns
df_excel[numeric_columns_excel] = df_excel[numeric_columns_excel].fillna(df_excel[numeric_columns_excel].median())

# Non-numeric columns
non_numeric_columns_excel = df_excel.select_dtypes(exclude=['number']).columns
df_excel[non_numeric_columns_excel] = df_excel[non_numeric_columns_excel].fillna(df_excel[non_numeric_columns_excel].mode().iloc[0])

# Convert Loan_Status to categorical
df_pdf['Loan_Status'] = df_pdf['Loan_Status'].map({'Y': 'Approved', 'N': 'Rejected'})

# Set LoanID as index



# Show the cleaned data
# Import the display function

# Show the cleaned data
display(df_pdf.head())
display(df_excel.head())



,Loan_ID,Gender,Married,Dependents,Graduate,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1002,1,0,0,1,0,5849,0,128,360,1,1,Approved
1,1003,1,1,1,1,0,4583,1508,128,360,1,3,Rejected
2,1005,1,1,0,1,1,3000,0,66,360,1,1,Approved
3,1006,1,1,0,0,0,2583,2358,120,360,1,1,Approved
4,1008,1,0,0,1,0,6000,0,141,360,1,1,Approved


,Loan_ID,Gender,Married,Dependents,Graduate,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,2284,1,0,0,0,0,3902,1666.0,109,333,1,3,Y
1,2287,2,0,0,1,0,1500,1800.0,103,333,0,2,N
2,2288,1,1,2,0,0,2889,0.0,45,180,0,1,N
3,2296,1,0,0,0,0,2755,0.0,65,300,1,3,N
4,2297,1,0,0,1,0,2500,20000.0,103,333,1,2,Y


In [51]:
# Descriptive analysis for df_excel
from IPython.display import display
# Total amount loaned by AFS
total_loan_amount_excel = df_pdf['LoanAmount'].sum()
print("Total amount loaned by AFS:", total_loan_amount_excel)

# Average amount loaned
average_loan_amount_excel = df_pdf['LoanAmount'].mean()
print("Average amount loaned:", average_loan_amount_excel)

# Average loan term
average_loan_term_excel = df_pdf['Loan_Amount_Term'].mean()
print("Average loan term:", average_loan_term_excel)

# Total number of applicants broken down into Approved and Rejected
approved_applicants_excel = df_pdf[df_pdf['Loan_Status'] == 'Approved'].shape[0]
rejected_applicants_excel = df_pdf[df_pdf['Loan_Status'] == 'Rejected'].shape[0]
print("Total number of applicants - Approved:", approved_applicants_excel)
print("Total number of applicants - Rejected:", rejected_applicants_excel)

# Number of males and females in each case
gender_counts_approved_excel = df_pdf[df_pdf['Loan_Status'] == 'Approved']['Gender'].value_counts()
gender_counts_rejected_excel = df_pdf[df_pdf['Loan_Status'] == 'Rejected']['Gender'].value_counts()
print("Gender distribution - Approved:")
print(gender_counts_approved_excel)
print("Gender distribution - Rejected:")
print(gender_counts_rejected_excel)

# Maximum and minimum loan amounts
max_loan_amount_excel = df_pdf['LoanAmount'].max()
min_loan_amount_excel = df_pdf['LoanAmount'].min()
print("Maximum loan amount:", max_loan_amount_excel)
print("Minimum loan amount:", min_loan_amount_excel)

# Number of self-employed who had their loan approved
self_employed_approved_excel = df_pdf[(df_pdf['Loan_Status'] == 'Approved') & (df_pdf['Self_Employed'] == 1)].shape[0]
total_approved_excel = df_pdf[df_pdf['Loan_Status'] == 'Approved'].shape[0]
self_employed_approved_percentage_excel = (self_employed_approved_excel / total_approved_excel) * 100
print("Percentage of self-employed who had their loan approved:", self_employed_approved_percentage_excel)

# Additional optional analysis
# Income distribution of all main applicants
income_distribution_excel = df_pdf['ApplicantIncome'].describe()
print("Income distribution of all main applicants:")
print(income_distribution_excel)

# Top ten applicants by loan amount
top_ten_applicants_excel = df_pdf.nlargest(10, 'LoanAmount')
print("Top ten applicants by loan amount:")
display(top_ten_applicants_excel)

# Distribution of properties (rural, urban, etc.) of all loan applicants
property_distribution_excel = df_pdf['Property_Area'].value_counts()
print("Distribution of properties of all loan applicants:")
display(property_distribution_excel)


Total amount loaned by AFS: 57780
Average amount loaned: 145.54156171284635
Average loan term: 344.43324937027705
Total number of applicants - Approved: 276
Total number of applicants - Rejected: 121
Gender distribution - Approved:
Gender
1    229
2     47
Name: count, dtype: int64
Gender distribution - Rejected:
Gender
1    97
2    24
Name: count, dtype: int64
Maximum loan amount: 700
Minimum loan amount: 17
Percentage of self-employed who had their loan approved: 11.956521739130435
Income distribution of all main applicants:
count      397.000000
mean      5349.604534
std       5728.624840
min        150.000000
25%       2957.000000
50%       3865.000000
75%       5695.000000
max      63337.000000
Name: ApplicantIncome, dtype: float64
Top ten applicants by loan amount:


,Loan_ID,Gender,Married,Dependents,Graduate,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
171,1585,1,1,3,1,0,51763,0,700,300,1,1,Approved
130,1469,1,0,0,1,1,20166,0,650,480,0,1,Approved
155,1536,1,1,3,1,0,39999,0,600,180,0,2,Approved
368,2191,1,1,0,1,0,19730,5266,570,360,1,3,Rejected
177,1610,1,1,3,1,0,5516,11300,495,360,0,2,Rejected
332,2101,1,1,0,1,0,63337,0,490,180,1,1,Approved
307,1996,1,0,0,1,0,20233,0,480,360,1,3,Rejected
278,1907,1,1,0,1,0,14583,0,436,360,1,2,Approved
371,2201,1,1,2,1,1,9323,7873,380,300,1,3,Approved
324,2067,1,1,1,1,1,8666,4983,376,360,0,3,Rejected


Distribution of properties of all loan applicants:


Property_Area
2    148
1    145
3    104
Name: count, dtype: int64